<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.  

**Цель работы:**  
Создать модель классификации комментариев на позитивные и негативные.  
Метрика качества - *F1*. Она должна быть не ниже 0.75.  

**Ход работы**
1. Импорт данных
2. Перевод текста в признак
    - Очистка текста
    - Лемматизация текста
    - Расчет TF-IDF
3. Обучение моделей 
4. Тестирование финальной модели

## Импорт библиотек

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Импорт данных

In [2]:
try:
    data_raw = pd.read_csv('toxic_comments.csv')
except:
    data_raw = pd.read_csv('/datasets/toxic_comments.csv')
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [3]:
data_raw.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Уберем лишний столбец *Unnamed: 0*

In [4]:
data_raw = data_raw.drop(['Unnamed: 0'], axis=1)

## Готовим признаки

### Лемматизация текста

Скачаем из библиотеки nltk нужные нам вещи(лемматизатор и стоп-слова)

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk_stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandrplakhotin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexandrplakhotin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Создадим функцию, которая:
 - Убирает из текса небуквенные символы, единичные буквы
 - Приводит каждое слово к его лемме

In [8]:
def clean_and_lem(text):
    txt = text.lower()
    txt = re.sub('[^a-z]', ' ', txt)
    txt = re.sub(r'\b\w{1,2}\b', ' ', txt)
    txt = txt.split()
    lem = WordNetLemmatizer()
    txt = [lem.lemmatize(word) for word in txt]
    txt = ' '.join(txt)
    return txt

In [9]:
data = data_raw.copy()
data['text'] = data['text'].apply(clean_and_lem)

Проверяем, что функция сработала верно

In [10]:
print(data['text'][0])
print('---')
print(data_raw['text'][0])

explanation why the edits made under username hardcore metallica fan were reverted they weren vandalism just closure some gas after voted new york doll fac and please don remove the template from the talk page since retired now
---
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


### TF-IDF

Преобразуем текст в понятный для моделей признак - вектор величины TF-IDF для каждого слова в тексте.

In [11]:
train, test = train_test_split(data, test_size=0.2, stratify=data['toxic'])
print(train.shape, test.shape)

(127433, 2) (31859, 2)


In [12]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
count_tf_idf.fit(train['text'])

X_train = count_tf_idf.transform(train['text'])
X_test = count_tf_idf.transform(test['text'])

y_train = train['toxic']
y_test = test['toxic']

## Обучение

После TfidfVectorizer мы получили признаки в виде разреженной матрицы(sparse matrix).  
Поэтому, возьмем модели, которые умеют с такой матрицей работать.  
Точность моделей будем проверять кросс-валидацией(5 фолдов)

### LogisticRegression

In [13]:
model = LogisticRegression(max_iter=1000, C=10)
cv = cross_val_score(model, X_train, y_train, scoring='f1', n_jobs=4)
print('Результат f1: {:.3f}'.format(cv.mean()))

Результат f1: 0.762


### LinearSVC

In [14]:
model = LinearSVC(random_state=123)
cv = cross_val_score(model, X_train, y_train, scoring='f1', n_jobs=4)
print('Результат f1: {:.3f}'.format(cv.mean()))

Результат f1: 0.768


### ComplementNB

In [15]:
model = ComplementNB(alpha=0.4)
cv = cross_val_score(model, X_train, y_train, scoring='f1', n_jobs=4)
print('Результат f1: {:.3f}'.format(cv.mean()))

Результат f1: 0.610


---
По итогам лучший результат на тренировочной выборке показала модель LinearSVC

## Тестирование

Узнаем результат на тестовой выборке

In [16]:
model = LinearSVC(random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Результат f1:{:.3f}'.format(f1_score(y_test, y_pred)))

Результат f1:0.779


## Выводы

Для задачи классификации комментариев на негативные и позитивные были обучены и протестированы несколько моделей.  
Обучались модели на размеченных комментариях, переведенных в матрицу векторов *TF-IDF*(оценки важности слова в контексте документа).  

Наилучший результат на тестовой выборке показала модель **LinearSVC** с **F1 = 0.77** (требуемый результат >0.75)